### Problem1: True, False, Uncertain with Explanation. Just guessing will earn you zero points.

a) in a multiple regression, if a regressor has a small p-value, then removing
it from the model will not cause the R2 to fall. 

False, removing any regressor could make the R2 fall. If it has a small p value that means it is statistically significant for the regression, and hence the R2 would definitely fall.

b) You want to estimate a model yi = α + βxi + ei . However, you are worried
that xi might be correlated with ei . One way to test this assumption is
to get the OLS residuals ˆei, and then check if cov(xi, ˆei) = 0. 


False this is not possible, there is no direct way to test the exogenity condition, and this covariance will always equal zero for the OLS estimator since it is a consequence of one of the first order conditions of univariate OLS.

(c) In a model with three regressors, X1, X2, and X3, the null hypothesis
H0 : β1 = β2 can be tested using a t-statistic on the transformed regression
with X1 −X2 as a new variable. 

This is true. Let us say that the new coeffecient on X1-X2 is B4. 
The new regression will be Y = B4(X1-X2) + B3X3. If we compare this to the orignal formulation of the regression Y = B1X1+B2X2+B3X3 then we may identify that testing whether B4=0 will tell us whether it is true that B1 = B2.

d) Consider the regression: You estimate the following regression:
Yi = β0 + β1 ·Malei + β2 ·Posti + β3 ·(Malei ×Posti) + ei, where Male is a dummy for male respondents, and Post is a dummy for whether the observation comes from after a policy change. You find that ˆβ3 = 2.0 and statistically significant. Therefore, you conclude that the policy increased Y by 2 units for males and had no effect for females.

This is false. For two reasons it's not true that the post has no effect for females providided this information B2 being non zero implies that there is an effect for females provided. Further the marginal effect of males of this policy would simply be B2+B3 not B3 alone. This can be seen by taking the differences for each of the four categorial groups in this study.

### Problem 2:

a) An increase of beatuy score of one unit increases the overall course rating out of 5 by .275 points. Further the coeffecient is definitely statistically significant. 

b) -.239-.253-.136+4.25=3.622

c) The only difference is the added effect of being male vs female. Hence, the difference between next sem and this sem is simply +0.239. The standard error of the difference would just be the standard error around the coeffecient female, which is 0.85. Hence 95% CI is [0.239-0.85*1.96,0.239+0.85*1.96] = [-1.427,1.905]

d) This could be because the one-credit course is an Omitted Variable that is positively correlated with beatuy of instructor and rating beause there was a positive omitted variable bias. It does make sense one credit courses tend to have different types of faculty that might be younger and therefore perceived as more attractive.

e)

Null: H_0: B on beauty and D interaction=0 (no change in slope when Beauty goes above 0; linear everywhere with slope .237).
Alt.: H_1: B on beauty and D interaction!=0 (a kink: marginal effect differs for above-average Beauty).

The t-stat is .081/.135 < 1.96 so clearly not significant.

f) here we have to use a difference in means test the difference in coeffecients for beauty in male and female subsets is .384-.128 = 0.252. The se of this test = (0.076^2+0.064^2)^(1/2) = 0.099 hence t-stat =  0.252/0.099 > 2 > 1.96, hence statistically signficiant at 5% level and there is a difference on the effect of beauty for course evlautions between men and women.


### Markdown 3

In [8]:
# Imports and data loading
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pathlib import Path

# Paths
DATA_PATH = Path('/Users/nikhileshbelulkar/Documents/Econometric-Data-Science/ps3/Growth.dta')

# Load data
_df = pd.read_stata(DATA_PATH)
print(f"Loaded shape: {_df.shape}")
print(_df.head(3))


Loaded shape: (65, 7)
  country_name    growth       rgdp60  tradeshare  yearsschool  rev_coups  \
0        India  1.915168   765.999817    0.140502         1.45   0.133333   
1    Argentina  0.617645  4462.001465    0.156623         4.99   0.933333   
2        Japan  4.304759  2953.999512    0.157703         6.71   0.000000   

   assasinations  
0       0.866667  
1       1.933333  
2       0.200000  


In [9]:
# Clean, filter, and construct variables

df = _df.copy()

# Try common country identifier column names present in Barro-Lee growth datasets
country_cols = [c for c in df.columns if c.lower() in ['country_name']]
print({'country_cols_found': country_cols})

# Exclude Malta if present
if country_cols:
    ccol = country_cols[0]
    before = len(df)
    df = df[df[ccol].str.upper().ne('MALTA')]
    after = len(df)
    print(f"Excluded Malta rows: {before - after}")
else:
    print("No explicit country name column found; assuming Malta not present as a separate identifier.")

# Harmonize variable names by lowercasing for safe access
cols_lower = {c: c.lower() for c in df.columns}
df = df.rename(columns=cols_lower)

# Identify/alias expected variables by common names in the assignment
# We will search for likely matches
candidate_map = {}

# growth variable candidates
for cand in ['growth', 'rgdpgrowth', 'gr6095', 'avg_grow', 'growthrate']:
    if cand in df.columns:
        candidate_map['growth'] = cand
        break

# trade share candidates
for cand in ['tradeshare', 'trade_share', 'openness', 'open', 'trade', 'trade60_95']:
    if cand in df.columns:
        candidate_map['tradeshare'] = cand
        break

# years of schooling candidates
for cand in ['yearsschool', 'yrs_school', 'yrs_schooling', 'school', 'schooling', 'avgyrs']:
    if cand in df.columns:
        candidate_map['yearsschool'] = cand
        break

# revolutions coups candidates
for cand in ['rev_coups', 'revcoups', 'revolutions', 'coups']:
    if cand in df.columns:
        candidate_map['rev_coups'] = cand
        break

# assassinations candidates (include common misspelling in the dataset)
for cand in ['assassinations', 'assasinations', 'assassin', 'assass', 'assas']:
    if cand in df.columns:
        candidate_map['assassinations'] = cand
        break

# RGDP 1960 level candidates
for cand in ['rgdp60', 'rgdp_1960', 'gdp60', 'gdp_1960', 'rgdppc60']:
    if cand in df.columns:
        candidate_map['rgdp60'] = cand
        break

print({'variable_mapping': candidate_map})

# Guard: ensure minimum required variables exist
required = ['growth', 'tradeshare', 'yearsschool']
missing = [k for k in required if k not in candidate_map]
if missing:
    raise ValueError(f"Missing required variables from dataset: {missing}. Columns available: {list(df.columns)[:40]} ...")

# Create working columns with standard names
w = {}
for std_name, col in candidate_map.items():
    w[std_name] = df[col]

work = pd.DataFrame(w)

# Optional controls
if 'rev_coups' in candidate_map:
    work['rev_coups'] = df[candidate_map['rev_coups']]
else:
    work['rev_coups'] = np.nan
if 'assassinations' in candidate_map:
    work['assassinations'] = df[candidate_map['assassinations']]
else:
    work['assassinations'] = np.nan
if 'rgdp60' in candidate_map:
    work['rgdp60'] = df[candidate_map['rgdp60']]
else:
    work['rgdp60'] = np.nan

# Transforms and interactions per specs
work['ln_yearsschool'] = np.log(work['yearsschool'])
work['ln_rgdp60'] = np.log(work['rgdp60']) if work['rgdp60'].notna().any() else np.nan
work['trade_ln_school'] = work['tradeshare'] * work['ln_yearsschool']
work['tradeshare2'] = work['tradeshare'] ** 2
work['tradeshare3'] = work['tradeshare'] ** 3

# Drop rows with missing needed variables on a model-by-model basis later; here just preview
work.head()


{'country_cols_found': ['country_name']}
Excluded Malta rows: 1
{'variable_mapping': {'growth': 'growth', 'tradeshare': 'tradeshare', 'yearsschool': 'yearsschool', 'rev_coups': 'rev_coups', 'assassinations': 'assasinations', 'rgdp60': 'rgdp60'}}


,growth,tradeshare,yearsschool,rev_coups,assassinations,rgdp60,ln_yearsschool,ln_rgdp60,trade_ln_school,tradeshare2,tradeshare3
0,1.915168,0.140502,1.45,0.133333,0.866667,765.999817,0.371564,6.641182,0.052205,0.019741,0.002774
1,0.617645,0.156623,4.99,0.933333,1.933333,4462.001465,1.607436,8.403353,0.251761,0.024531,0.003842
2,4.304759,0.157703,6.71,0.000000,0.200000,2953.999512,1.903599,7.990915,0.300204,0.024870,0.003922
3,2.930097,0.160405,2.89,0.100000,0.100000,1783.999878,1.061257,7.486613,0.170231,0.025730,0.004127
4,1.712265,0.160815,8.66,0.000000,0.433333,9895.003906,2.158715,9.199785,0.347154,0.025861,0.004159


In [10]:
# Run the five OLS regressions with HC1 robust SEs

models = {}
results = {}

# Helper to fit with HC1 robust SEs and drop NA rows per formula

def fit_formula(formula: str):
    data = work.copy()
    mod = smf.ols(formula, data=data)
    res = mod.fit(cov_type='HC1')
    return res

# (1) Growth ~ TradeShare + YearsSchool
models['(1)'] = 'growth ~ tradeshare + yearsschool'
results['(1)'] = fit_formula(models['(1)'])

# (2) Growth ~ TradeShare + ln(YearsSchool)
models['(2)'] = 'growth ~ tradeshare + ln_yearsschool'
results['(2)'] = fit_formula(models['(2)'])

# (3) Growth ~ TradeShare + ln(YearsSchool) + Rev coups + Assassinations + ln(RGDP60)
controls = []
if work['rev_coups'].notna().any():
    controls.append('rev_coups')
if work['assassinations'].notna().any():
    controls.append('assassinations')
if work['ln_rgdp60'].notna().any():
    controls.append('ln_rgdp60')

rhs3 = ' + '.join(['tradeshare', 'ln_yearsschool'] + controls)
models['(3)'] = f"growth ~ {rhs3}"
results['(3)'] = fit_formula(models['(3)'])

# (4) Add interaction TradeShare × ln(YearsSchool)
models['(4)'] = models['(3)'] + ' + trade_ln_school'
results['(4)'] = fit_formula(models['(4)'])

# (5) Polynomial in TradeShare up to cubic plus controls
poly_terms = ['tradeshare', 'tradeshare2', 'tradeshare3']
rhs5 = ' + '.join(poly_terms + ['ln_yearsschool'] + controls)
models['(5)'] = f"growth ~ {rhs5}"
results['(5)'] = fit_formula(models['(5)'])

# Quick text summaries (optional to view)
for k in ['(1)', '(2)', '(3)', '(4)', '(5)']:
    print(f"Model {k}: {models[k]}")
    print(results[k].summary().tables[1])
    print()

'fit complete'


Model (1): growth ~ tradeshare + yearsschool
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.1222      0.691     -0.177      0.860      -1.477       1.232
tradeshare      1.8978      0.866      2.193      0.028       0.201       3.594
yearsschool     0.2430      0.076      3.202      0.001       0.094       0.392

Model (2): growth ~ tradeshare + ln_yearsschool
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.1857      0.566     -0.328      0.743      -1.295       0.924
tradeshare         1.7490      0.794      2.204      0.028       0.194       3.304
ln_yearsschool     1.0163      0.204      4.974      0.000       0.616       1.417

Model (3): growth ~ tradeshare + ln_yearsschool + rev_coups + assassinations + ln_rgdp60
 

'fit complete'

In [11]:
# Build a combined regression results table similar to S&W Table 7.1

def collect_table(results_dict):
    rows = []
    # Collect coefficient and robust SE for key regressors across models
    keys = ['tradeshare', 'yearsschool', 'ln_yearsschool', 'rev_coups', 'assassinations', 'ln_rgdp60', 'trade_ln_school', 'tradeshare2', 'tradeshare3']
    for k in results_dict:
        res = results_dict[k]
        params = res.params
        ses = res.bse  # robust SEs due to cov_type='HC1'
        nobs = int(res.nobs)
        r2 = res.rsquared
        for name in keys:
            if name in params.index:
                rows.append({'var': name, 'model': k, 'coef': params[name], 'se': ses[name]})
        rows.append({'var': '_nobs', 'model': k, 'coef': nobs, 'se': np.nan})
        rows.append({'var': '_r2', 'model': k, 'coef': r2, 'se': np.nan})
    tab = pd.DataFrame(rows)
    return tab

_tab = collect_table(results)

# Pivot to a readable wide format: for each variable, show coef (se) under each model
out_rows = []
for var in sorted(set(_tab['var'])):
    if var.startswith('_'):
        continue
    line = {'Variable': var}
    for m in ['(1)', '(2)', '(3)', '(4)', '(5)']:
        sub = _tab[(_tab['var'] == var) & (_tab['model'] == m)]
        if len(sub):
            c = sub.iloc[0]['coef']
            se = sub.iloc[0]['se']
            line[m] = f"{c:.3f} ({se:.3f})"
        else:
            line[m] = ''
    out_rows.append(line)

# Add N and R^2 rows
for meta, label in [('_nobs', 'N'), ('_r2', 'R^2')]:
    line = {'Variable': label}
    for m in ['(1)', '(2)', '(3)', '(4)', '(5)']:
        sub = _tab[(_tab['var'] == meta) & (_tab['model'] == m)]
        if len(sub):
            val = sub.iloc[0]['coef']
            line[m] = f"{val:.0f}" if meta == '_nobs' else f"{val:.3f}"
        else:
            line[m] = ''
    out_rows.append(line)

results_table = pd.DataFrame(out_rows)
results_table


,Variable,(1),(2),(3),(4),(5)
0,assassinations,,,0.228 (0.335),0.224 (0.337),0.102 (0.364)
1,ln_rgdp60,,,-1.621 (0.440),-1.641 (0.440),-1.584 (0.477)
2,ln_yearsschool,,1.016 (0.204),2.161 (0.400),2.525 (0.639),2.133 (0.416)
3,rev_coups,,,-2.300 (0.921),-2.350 (0.921),-2.035 (0.938)
4,trade_ln_school,,,,-0.690 (0.809),
5,tradeshare,1.898 (0.866),1.749 (0.794),1.104 (0.748),1.883 (1.344),-5.702 (8.183)
6,tradeshare2,,,,,8.488 (15.829)
7,tradeshare3,,,,,-2.760 (8.527)
8,yearsschool,0.243 (0.076),,,,
9,N,64,64,64,64,64


(a) Regressions (1) and (2) differ in their treatment of YearsSchool. Based
on the evidence available to you, which specification is preferable? Briefly
explain.

ln(yearschool) is the better treatment, because the coefficent is more statistically significant than yearsschool (higher t-stat 4.98 vs 3.20) and has a higher R^2 value too. 

In [12]:
# Joint significance test for higher powers of TradeShare in model (5)

res5 = results['(5)']
H = 'tradeshare2 = 0, tradeshare3 = 0'

wald_f = res5.wald_test(H, use_f=True)      # robust Wald F

print('Testing H0: tradeshare2 = 0 and tradeshare3 = 0 (joint)')
print('\nRobust Wald F:')
print(wald_f)

pval = float(wald_f.pvalue)
alpha = 0.10
print(f"\nConclusion at {alpha*100:.0f}% level: {'Reject' if pval < alpha else 'Fail to reject'} H0.")


Testing H0: tradeshare2 = 0 and tradeshare3 = 0 (joint)

Robust Wald F:
<F test: F=array([[1.96317777]]), p=0.1499562227730685, df_denom=56, df_num=2>

Conclusion at 10% level: Fail to reject H0.


/Users/nikhileshbelulkar/opt/miniconda3/envs/fingers_crossed/lib/python3.11/site-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


(b) The effect on growth of expanding trade is the same no matter what the level of trade share is.

At the 10% level it is true that the other polynomials are not jointly significant so therefore it is plausible that the effect on growth of expanding trade changes with level of trade share.

c) To obtain the growth benefits of trade requires an educated work force.
A more educated work force is more able to use new technologies, so that
trade share has a larger effect on growth

This is not true, first of all this would be signified by the interaction ln_school*trade which is negative and insigificant. So no it is not necessarily better for growth to have amore educated workforce able to use new technologies.

d) Although trade is correlated with growth, this effect is a spurious consequence of political instability (which can be measured by number of revolutions and number of political assassinations). Political instability reduces both economic growth and trade, so the relationship between growth and
trade is spurious

Yes the relationship between growth and trade could be suprious as a consequence of the OMVB of political instability (proxied by assinations and revoutions). You can see that revolutions is statistically significant and further that the coeffecient on trade becomes not statisically significant in regressions 3-5 as there would be a reduction in the positive bias provided by polticial instability regressor variables (negatively correlated with trade and with growth therefore imposing a positive bias on the coefficient).

e) The effect of trade on growth is so small as to be unimportant in a practical sense. An economy could go from a trade share of 10% to a trade share of 50% with only a negligible effect on real GDP.


In the full regression all the coeffcients on tradeshare are statistically insignificant suggesting that they are indistinguishable from zero indicating that there is a negligible impact on real GDP.